## Introduction to Linear Regression
In this , we'll discuss one of the foundational algorithms in machine learning: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region. Here's the training data:

![title](lrtable.png)

linear-regression-training-data

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

In [20]:
import numpy as np
import torch

We can represent the training data using two matrices: inputs and targets, each with one row per observation, and one column per variable.

In [21]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [22]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [23]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges

In [24]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.7696, -1.4916,  0.1087],
        [-1.0678,  0.0797,  0.1165]], requires_grad=True)
tensor([ 0.4896, -1.3496], requires_grad=True)


torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).



![title](matrix_weight.png)

In [25]:
def model(x):
    return x @ w.t() + b

@ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [26]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -38.5916,  -68.9555],
        [ -53.7797,  -84.0576],
        [-126.1227,  -76.8212],
        [  18.8729, -102.5329],
        [ -81.9917,  -59.2295]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [27]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

Calculate the difference between the two matrices (preds and targets).

Square all elements of the difference matrix to remove negative values.

Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).

In [28]:
## MSE loss
def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()


In [29]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(27024.5059, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True. We'll see how this is useful in just a moment.

In [30]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [31]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.7696, -1.4916,  0.1087],
        [-1.0678,  0.0797,  0.1165]], requires_grad=True)
tensor([[-10715.8428, -13787.6738,  -7995.1162],
        [-14353.7246, -15364.2158,  -9525.4375]])


In [32]:
print(b)
print(b.grad)

tensor([ 0.4896, -1.3496], requires_grad=True)
tensor([-132.5226, -170.3193])


## Adjust weights and biases to reduce the loss

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's slope w.r.t. the weights and biases.

If a gradient element is positive:

increasing the weight element's value slightly will increase the loss

decreasing the weight element's value slightly will decrease the loss

![title](loss1.png)

If a gradient element is negative:

increasing the weight element's value slightly will decrease the loss</p>
decreasing the weight element's value slightly will increase the loss

![title](loss2.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the gradient descent optimization algorithm that we'll use to improve our model (by descending along the gradient).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [33]:
print(w.grad)
print(b.grad)

tensor([[-10715.8428, -13787.6738,  -7995.1162],
        [-14353.7246, -15364.2158,  -9525.4375]])
tensor([-132.5226, -170.3193])


In [34]:
print(w)
w.grad

tensor([[ 0.7696, -1.4916,  0.1087],
        [-1.0678,  0.0797,  0.1165]], requires_grad=True)


tensor([[-10715.8428, -13787.6738,  -7995.1162],
        [-14353.7246, -15364.2158,  -9525.4375]])

In [44]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [45]:
# Let's verify that the loss is actually lower
# Generate predictions
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(18812.6797, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [39]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

Let's implement the above step by step.

In [46]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [47]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(534.1873, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [49]:
preds

tensor([[ 67.3911,  70.0797],
        [ 90.4013,  98.7422],
        [ 83.6182, 137.7077],
        [ 80.1679,  36.8065],
        [ 81.7923, 115.6414]], grad_fn=<AddBackward0>)

In [51]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [52]:
import torch.nn as nn

In [53]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [54]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [55]:
from torch.utils.data import TensorDataset

In [56]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [57]:
from torch.utils.data import DataLoader

In [58]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [60]:
for x,y in train_dl:
    print(x,y)
    print("*"*80)

tensor([[102.,  43.,  37.],
        [103.,  43.,  36.],
        [ 73.,  66.,  44.],
        [ 73.,  67.,  43.],
        [ 92.,  87.,  64.]]) tensor([[ 22.,  37.],
        [ 20.,  38.],
        [ 57.,  69.],
        [ 56.,  70.],
        [ 82., 100.]])
********************************************************************************
tensor([[ 91.,  88.,  64.],
        [101.,  44.,  37.],
        [ 74.,  66.,  43.],
        [ 87., 135.,  57.],
        [ 88., 134.,  59.]]) tensor([[ 81., 101.],
        [ 21.,  38.],
        [ 57.,  69.],
        [118., 134.],
        [118., 132.]])
********************************************************************************
tensor([[ 87., 134.,  58.],
        [ 68.,  97.,  70.],
        [ 68.,  96.,  71.],
        [ 69.,  96.,  70.],
        [ 91.,  87.,  65.]]) tensor([[119., 133.],
        [102., 120.],
        [104., 118.],
        [103., 119.],
        [ 80., 102.]])
********************************************************************************


Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [61]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4047,  0.1842, -0.3080],
        [ 0.5366,  0.2948,  0.1439]], requires_grad=True)
Parameter containing:
tensor([0.2594, 0.0721], requires_grad=True)


In [66]:
print(model.parameters)

<bound method Module.parameters of Linear(in_features=3, out_features=2, bias=True)>


In [67]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-30.1865,  65.1855],
        [-40.0713,  84.0577],
        [-28.1306,  94.6094],
        [-44.4954,  72.8081],
        [-31.5426,  75.4743],
        [-30.7754,  65.4273],
        [-40.5635,  83.9068],
        [-28.8433,  95.2899],
        [-43.9065,  72.5663],
        [-31.4460,  75.0816],
        [-30.6787,  65.0346],
        [-40.6602,  84.2995],
        [-27.6383,  94.7603],
        [-44.5920,  73.2008],
        [-30.9537,  75.2325]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [68]:
# Import nn.functional
import torch.nn.functional as F

In [69]:
# Define loss function
loss_fn = F.mse_loss

In [71]:
loss = loss_fn(model(inputs),targets)

In [72]:
loss

tensor(7104.7202, grad_fn=<MseLossBackward>)

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [73]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.



In [75]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    count = 0
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            count +=1
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print(count)


Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [76]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 243.5281
Epoch [20/100], Loss: 231.5603
Epoch [30/100], Loss: 118.5777
Epoch [40/100], Loss: 54.8612
Epoch [50/100], Loss: 82.0685
Epoch [60/100], Loss: 46.1749
Epoch [70/100], Loss: 37.4359
Epoch [80/100], Loss: 84.8971
Epoch [90/100], Loss: 52.0941
Epoch [100/100], Loss: 68.2930
300


In [77]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.1784,  72.1674],
        [ 79.3146,  98.3130],
        [123.7951, 136.4879],
        [ 26.2331,  46.4681],
        [ 93.9234, 109.4507],
        [ 56.9577,  71.2178],
        [ 78.5762,  97.8596],
        [123.7899, 136.8436],
        [ 27.4537,  47.4177],
        [ 94.4057, 109.9470],
        [ 57.4400,  71.7141],
        [ 78.0939,  97.3634],
        [124.5334, 136.9413],
        [ 25.7508,  45.9718],
        [ 95.1441, 110.4004]], grad_fn=<AddmmBackward>)

In [78]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])